In [ ]:
!pip install GarminConnect

import numpy as np
import pandas as pd
import plotly.graph_objects as go
from datetime import date, datetime
from garminconnect import Garmin

#login
from getpass import getpass

email = input("Enter email address: ")
password = getpass("Enter password: ")

garmin = Garmin(email, password)
garmin.login()

In [ ]:

#save session
import os

GARTH_HOME = os.getenv("GARTH_HOME", "~/.garth")
garmin.garth.dump(GARTH_HOME)


#Enter the date you want to pull data for
year = input("Enter the year of the day you wish to graph e.g. 2023 ")
month = input("Enter the month number of the day you wish to graph e.g. 9 for September ")
day = input("Enter the month date of the day you wish to graph e.g. 15 for the 15th of the month ")
max_hr = input("What is your max heart rate? ")
for_date = date(int(year), int(month), int(day))

In [ ]:
#Pull HR data from that date
heart_rate_data = garmin.get_heart_rates(for_date.isoformat())
heart_rate_list = [i[1] for i in heart_rate_data['heartRateValues']]
timestamp_list = [datetime.fromtimestamp(i[0]/1000) for i in heart_rate_data['heartRateValues']]

#Build a dataframe of the returned data so I can filter out sleep time
df = pd.DataFrame({'timestamp': timestamp_list, 'heart_rate': heart_rate_list})
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['local_time'] = df['timestamp'].dt.tz_localize('utc').dt.tz_convert('US/Mountain') #Change to appropriate time zone if not U.S. Mountaim
start = f"{year}-{month}-{day} 06:00:00" #Change start and end times from 06:00 to whatever time you wake up and 22:00 to whatever time you go to sleep
end = f"{year}-{month}-{day} 22:00:00"
df = df[df['local_time'] > start] #Waking Hours Between 6 am and 10 pm
df = df[df['local_time'] < end]

#Return the new waking HR list
heart_rate_list = df['heart_rate'].tolist()

#Create bins based on max HR
max_hr = float(max_hr)
rest = max_hr * 0.4
stand = max_hr * 0.5
z0 = max_hr * 0.6
z1 = max_hr * 0.7
z2 = max_hr * 0.8
z3_4 = max_hr * 0.9
z5 = max_hr

#Set the distribution bins that I want to look at for zones (yours might be different)
hist, bins = np.histogram(heart_rate_list, bins=[0, rest, stand, z0, z1, z2, z3_4, z5])
hist = [(i * 2)/60 for i in hist] #2 min samples
bin_names = ['Sit/Relax: <40% MHR', 'Stand/Move: 40-50% MHR', 'Z0 Exercise: 50-60% MHR', 'Z1 Exercise: 60-70% MHR', 'Z2 Exercise: 70-80% MHR', 'Z3/4 Exercise: 80-90% MHR', 'Z5+: >90% MHR']

#Build the chart
fig = go.Figure()
fig.add_trace(go.Bar(x=bin_names, y=hist, marker=dict(color=['red','blue','blue','blue','blue','blue','blue'])))
fig.update_layout(title_text=f'Daily Waking Heart Rate Zone Distribution ({month}/{day}/{year})', font=dict(size=18), yaxis_title='Hours')
fig.show()